In [19]:
# hard coded path (TODO: see how it can be NOT hard coded)
pTodoistToken = '/Volumes/GoogleDrive/My Drive/Code_PGDrive/api_todoist/todoist_api_token.txt'
pGoogleSheet = '/Volumes/GoogleDrive/My Drive/Code_PGDrive/api_google/credentials.json'

In [20]:
# import libaries
import os
import numpy as np
import pandas as pd
from todoist_api_python.api import TodoistAPI
import datetime
import gspread

In [90]:
# get recurrent event titles from google sheet
sa = gspread.service_account(filename=pGoogleSheet)
sh = sa.open_by_key('1E-6rLfbm18oCs5Iha5ticZshvlzG2RVOg_FSWJS8iOQ')
wks = sh.worksheet("Sheet1")
dfwks = pd.DataFrame(wks.get_all_records())

# clean
# remove brackets
df = dfwks.title.str.rpartition(sep=' (', expand=True)
dfwks['title_clean'] = df.loc[:,0].values

In [91]:
i = 0
# create single tasks from the spread sheet of the same name but without the front emoji
# content: from "☀️ Morning Routine (81)" to "Morning Routine [60m]" 
# TODO: remove emoji
title = dfwks.title_clean[i]
title

# start date and time
# label corresponds to the color code (need to create color label corresponds)
# priority = 4
# send to todoist for creating new task

'☀️ Morning Routine'

In [4]:
# log on to todoist
with open (pTodoistToken) as f:
    todoist_api = f.readlines()
    todoist_api = todoist_api[0]

Handling recurrent tasks move to next recurrent event


In [50]:
# get a recurrent task to see the values
api = TodoistAPI(todoist_api)
try:
    todoistTasks = api.get_tasks()
except Exception as error:
    print(error)
# put in dictionaries
c = []
todoId = []
ind = []
tskTitleId = {}
tskIdInd = {}
for i in range(len(todoistTasks)):
    c.append(todoistTasks[i].content)
    todoId.append(todoistTasks[i].id)
    ind.append(i)
    tskTitleId[todoistTasks[i].content] = todoistTasks[i].id
    tskIdInd[todoistTasks[i].id] = i
dfTodoRef = pd.DataFrame()
dfTodoRef['ind'] = ind
dfTodoRef['id'] = todoId
dfTodoRef['content'] = c


In [59]:
# check if more than one task with the same name
tskName = dfwks.title[0]
if sum(dfTodoRef['content'] == tskName) != 1:
    raise Exception(f'ERROR: more than 1 task with the same name: {tskName}')
else:
    print(f'VALIDATED: only 1 task with this name: {tskName}')


VALIDATED: only 1 task with this name: ☀️ Morning Routine (81)


In [ ]:
# eventTitle = dfwks.title[0]
taskinfo = todoistTasks[tskIdInd[tskTitleId[dfwks.title[0]]]]
taskinfo